In [ ]:
import json
import re
import re
from nltk.tokenize import TweetTokenizer
import string
import demoji
import pickle

In [ ]:
month = "nov"
data_type = "reddit"
input_file = "selected_discussion_" + month + ".jsonlist"
output_text_file = "text_clean_" + data_type + ".pkl"
output_com_file = "count_clean_" + data_type + ".pkl"

with open(input_file, "r") as f:
    data = f.readlines()
n = len(data)
print(n)

In [ ]:
def removeUsernames(string):
    return re.sub('@[^\s]+', '@user', string)


def specialUnicodeRemover(string):
    return re.sub(r"(\xe9|\362)", "", string)


def punctuationRemover(tweet):
    ls = list(string.punctuation)
    ls.remove('@')
    try:
        tweet = tweet.translate(str.maketrans('', '', ls))
        return tweet
    except:
        return tweet


def RTRemover(string):
    string = string.strip()
    if 'RT' in string[0:2]:
        string = string[2:]
        return string
    else:
        return string


def EmojiRemover(string):
    return demoji.replace(string, "")


def DotRemover(string):
    if '...' in string:
        string = string.replace('...', '')
    elif '..' in string:
        string = string.replace('..', '')
    return string

In [ ]:
text_final = []
num_comments_final = []
for i in range(n):
    print("################")
    print(i)
    each_reddit = json.loads(data[i])
    value = list(each_reddit.keys())[0]
    each_reddit = each_reddit[value]
    for thread in each_reddit:
        num_comments = thread.get('comments', [])
        num_comments = len(num_comments)
        if num_comments == 0:
            continue
        raw_text = thread.get('selftext', '') + ' ' + thread.get('title', '')
        raw_text = raw_text.strip()
        if raw_text:
            cleaned_text = RTRemover(raw_text)
            cleaned_text = removeUsernames(raw_text)
            cleaned_text = re.sub(r'https?:\/\/\S*',
                                  '@url',
                                  cleaned_text,
                                  flags=re.MULTILINE)
            cleaned_text = specialUnicodeRemover(cleaned_text)
            cleaned_text = punctuationRemover(cleaned_text)
            cleaned_text = EmojiRemover(cleaned_text)
            cleaned_text = DotRemover(cleaned_text)
            if cleaned_text:
                cleaned_text = cleaned_text.lower()
                text_final.append(cleaned_text)
                num_comments_final.append(num_comments)

In [ ]:
assert len(text_final) == len(num_comments_final)
print(len(text_final))
with open(output_text_file, "wb") as f:
    pickle.dump(text_final, f)
with open(output_com_file, "wb") as f:
    pickle.dump(num_comments_final, f)